<a href="https://colab.research.google.com/github/mjgpinheiro/Econophysics/blob/main/ML_Prophet_stocks2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install ta
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
import ta
from prophet import Prophet
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso


# Download data
data = yf.download('AAPL', start='2016-01-01', end='2023-03-26')
if data.empty:
    raise ValueError("No data found for the specified dates.")
prices = data['Close'].to_numpy()

# Add technical indicators
rsi = ta.momentum.RSIIndicator(data['Close'], window=14)
data['RSI'] = rsi.rsi()

stoch = ta.momentum.StochasticOscillator(high=data['High'], low=data['Low'], close=data['Close'], window=14, smooth_window=3)
data['%K'] = stoch.stoch()


def predict(prices):
    # Split data into training and testing sets
    train_prices = prices[:-365]
    test_prices = prices[-365:]
    if len(train_prices) < 5:
        return ["hold"] * len(test_prices), 0
    print(train_prices,test_prices)

    # Scale the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(train_prices.reshape(-1, 1))

    # Add RSI and Stochastic Oscillator to training data
    train_rsi = data['RSI'][:-365].to_numpy().reshape(-1, 1)
    train_k_percent = data['%K'][:-365].to_numpy().reshape(-1, 1)
    X_train = np.concatenate((X_train, train_rsi, train_k_percent), axis=1)

    # Use grid search to find the best hyperparameters
    if len(train_prices) >= 5:
        param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10]}
        # Impute missing values with the mean
        imputer = SimpleImputer(strategy='mean')
        X_train = imputer.fit_transform(X_train)

        grid_search = GridSearchCV(Lasso(), param_grid, cv=5)
        grid_search.fit(X_train[:-1], X_train[1:])
        best_alpha = grid_search.best_params_['alpha']
    else:
        best_alpha = 0.001

    # Train the model
    model = Sequential()
    model.add(LSTM(128, input_shape=(1, 3)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(X_train[:-1].reshape(-1, 1, 3), X_train[1:], epochs=50, batch_size=1, verbose=0)

# Make prediction with LSTM model
    X_test = scaler.transform(test_prices.reshape(-1, 1))
    test_rsi = data['RSI'][-365:].to_numpy().reshape(-1, 1)
    test_k_percent = data['%K'][-365:].to_numpy().reshape(-1, 1)
    X_test = np.concatenate((X_test, test_rsi, test_k_percent), axis=1)
    y_pred = model.predict(X_test.reshape(-1, 1, 3))
    y_pred = scaler.inverse_transform(y_pred).flatten()

    daily_returns = np.diff(prices[-365:]) / prices[-365:-1]
    strategy_returns = np.diff(y_pred) / prices[-365:-1]

# Calculate benchmark return
    benchmark_prices = data['Open'].to_numpy()[-365:]
    benchmark_returns = np.diff(benchmark_prices) / benchmark_prices[:-1]
    benchmark_avg_return = np.mean(benchmark_returns)

# Print average return and Sharpe ratio for strategy and benchmark
    print("Average daily return (strategy):", np.mean(strategy_returns))
    print("Sharpe ratio (strategy):", np.sqrt(365) * np.mean(strategy_returns - daily_returns) / np.std(strategy_returns - daily_returns))
    print("Average daily return (benchmark):", benchmark_avg_return)
    print("Sharpe ratio (benchmark):", np.sqrt(365) * np.mean(benchmark_returns - daily_returns) / np.std(benchmark_returns - daily_returns))

# Plot actual and predicted prices
    plt.plot(prices[-365:], label='Actual')
    plt.plot(y_pred, label='Predicted')
    plt.plot(benchmark_prices, label='Benchmark')
    plt.legend()
    plt.show()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
[*********************100%***********************]  1 of 1 completed
